In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install split-folders
#!pip install tflearn
#!pip install helpers
!pip install keras-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.4 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=dc4cf9c1146a9d8ba7c09b18e7723473399dff195347a0b36159261910e1b376
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [12]:
import os
from random import shuffle
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
import matplotlib
matplotlib.use("Agg")
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

In [4]:
TRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/Train_Data/Train'

In [5]:
def create_label(image_name):
    word_label = image_name.split('.')[0]
    label = word_label.split('(')[0]
    if label == 'Acne-Rosacea ':  # 0
        return np.array([1 , 0 , 0 , 0])
    elif label == 'Eczema ':  # 1
        return np.array([0 , 1 , 0 , 0])
    elif label == 'Psoriasis-Relatives ':  # 2
        return np.array([0 , 0 , 1 , 0])
    elif label == 'Vitiligo ':  # 3
        return np.array([0 , 0 , 0 , 1])



In [6]:
import splitfolders
train_paths='/content/drive/MyDrive/Colab Notebooks/Train_Data'
splitfolders.ratio(train_paths, output="Output", seed=1337, ratio=(.8, .2), group_prefix=None)

Copying files: 2636 files [00:52, 49.89 files/s] 


In [7]:
train_path='/content/Output/train/Train'
valid_path='/content/Output/val/Train'
size=64
def create_train_data():
    training_data = []
    for img in os.listdir(train_path):
        path = os.path.join(train_path, img)
        img_data = cv2.imread(path,3)
        image_resize = cv2.resize(img_data,(size,size))
        image=cv2.cvtColor(image_resize, cv2.COLOR_BGR2RGB)
        training_data.append([np.array(image), create_label(img)])
    shuffle(training_data)
    np.save('trainings_data.npy', training_data)
    return training_data

def create_valid_data():
    validing_data=[]
    for img in os.listdir(valid_path):
        path = os.path.join(valid_path, img)
        img_data = cv2.imread(path,3)
        image_resize = cv2.resize(img_data,(size,size))
        image=cv2.cvtColor(image_resize, cv2.COLOR_BGR2RGB)
        validing_data.append([np.array(image), create_label(img)])
    shuffle(validing_data)
    np.save('validating_data.npy', validing_data)
    return validing_data
if (os.path.exists('training_data.npy')): # If you have already created the dataset:
    train_data =np.load('training_data.npy',allow_pickle=True)
else: # If dataset is not created:
    train_data = create_train_data()

#print(train_data.shape)
if (os.path.exists('validating_data.npy')):
    val_data =np.load('validating_data.npy',allow_pickle=True)
else:
    val_data = create_valid_data()



/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [13]:

train = train_data
valid = val_data
num_classes=4
Xtrain = np.array([i[0] for i in train]).reshape(-3,size,size,3)
ytrain = np.array([i[1] for i in train])
X_train=np.asarray(Xtrain).astype(np.float32)
y_train=np.asarray(ytrain).astype(np.float32)


Xvalid = np.array([i[0] for i in valid]).reshape(-3,size,size,3)
yvalid = np.array([i[1] for i in valid])
X_valid=np.asarray(Xvalid).astype(np.float32)
y_valid=np.asarray(yvalid).astype(np.float32)
X_train = X_train / 255.0
X_valid = X_valid / 255.0
mean = np.mean(X_train, axis=0)
X_train -= mean
X_valid -= mean
                                # Step 2
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_valid = lb.transform(y_valid)


In [ ]:
X_train.shape

(2108, 64, 64, 3)

In [14]:
def conv_module(input,No_of_filters,filtersizeX,filtersizeY,stride,chanDim,padding="same"):
  input = Conv2D(No_of_filters,(filtersizeX,filtersizeY),strides=stride,padding=padding)(input)
  input = BatchNormalization(axis=chanDim)(input)
  input = Activation("relu")(input)
  return input

In [15]:
def inception_module(input,numK1x1,numK3x3,numk5x5,numPoolProj,chanDim):
                                 #Step 1
  conv_1x1 = conv_module(input, numK1x1, 1, 1,(1, 1), chanDim)
                                 #Step 2
  conv_3x3 = conv_module(input, numK3x3, 3, 3,(1, 1), chanDim)
  conv_5x5 = conv_module(input, numk5x5, 5, 5,(1, 1), chanDim)
                                 #Step 3
  pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input)
  pool_proj = Conv2D(numPoolProj, (1, 1), padding='same', activation='relu')(pool_proj)
                                 #Step 4
  input = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=chanDim)
  return input

In [16]:
def downsample_module(input,No_of_filters,chanDim):
  conv_3x3=conv_module(input,No_of_filters,3,3,(2,2),chanDim,padding="valid")
  pool = MaxPooling2D((3,3),strides=(2,2))(input)
  input = concatenate([conv_3x3,pool],axis=chanDim)
  return input

In [17]:
def GoogleNet(width,height,depth,classes):
  inputShape=(height,width,depth)
  chanDim=-1

  # (Step 1) Define the model input
  inputs = Input(shape=inputShape)

  # First CONV module
  x = conv_module(inputs, 96, 3, 3, (1, 1),chanDim)

  # (Step 2) Two Inception modules followed by a downsample module
  x = inception_module(x, 32, 32,32,32,chanDim)
  x = inception_module(x, 32, 48, 48,32,chanDim)
  x = downsample_module(x, 80, chanDim)

  # (Step 3) Five Inception modules followed by a downsample module
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = inception_module(x, 96, 64, 32,32,chanDim)
  x = inception_module(x, 80, 80, 32,32,chanDim)
  x = inception_module(x, 48, 96, 32,32,chanDim)
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = downsample_module(x, 96, chanDim)

  # (Step 4) Two Inception modules followed
  x = inception_module(x, 176, 160,96,96, chanDim)
  x = inception_module(x, 176, 160, 96,96,chanDim)

  # Global POOL and dropout
  x = AveragePooling2D((7, 7))(x)
  x = Dropout(0.8)(x)

  # (Step 5) Softmax classifier
  x = Flatten()(x)
  x = Dense(classes)(x)
  x = Activation("softmax")(x)

  # Create the model
  model = Model(inputs, x, name="googlenet")
  return model


In [18]:
NUM_EPOCHS = 100
INIT_LR = 5e-3
def poly_decay(epoch):
  maxEpochs = NUM_EPOCHS
  baseLR = INIT_LR
  power = 1.0
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  return alpha

In [19]:
aug = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,vertical_flip=True,fill_mode="nearest")

In [20]:
callbacks=[LearningRateScheduler(poly_decay)]
opt = SGD(lr=INIT_LR, momentum=0.9)
model = GoogleNet(width=size, height=size, depth=3, classes=4)
                                    # Step 1
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
                                    # Step 2
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 96)   2688        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 96)  384         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 64, 64, 96)   0           ['batch_normalization[0][

In [ ]:
model.fit(aug.flow(X_train, y_train, batch_size=64),validation_data=(X_valid, y_valid), steps_per_epoch=50,epochs=NUM_EPOCHS, callbacks=callbacks, verbose=1)

Epoch 1/100
50/50 [==============================] - ETA: 0s - batch: 24.5000 - size: 63.9200 - loss: 1.3948 - acc: 0.5078

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50/50 [==============================] - 66s 783ms/step - batch: 24.5000 - size: 63.9200 - loss: 1.3948 - acc: 0.5078 - val_loss: 1.4134 - val_acc: 0.3883 - lr: 0.0050
Epoch 2/100
50/50 [==============================] - 27s 536ms/step - batch: 24.5000 - size: 63.8400 - loss: 1.0282 - acc: 0.6071 - val_loss: 1.4801 - val_acc: 0.3580 - lr: 0.0049
Epoch 3/100
50/50 [==============================] - 28s 569ms/step - batch: 24.5000 - size: 63.9200 - loss: 0.8794 - acc: 0.6652 - val_loss: 1.7434 - val_acc: 0.3220 - lr: 0.0049
Epoch 4/100
50/50 [==============================] - 27s 548ms/step - batch: 24.5000 - size: 63.8400 - loss: 0.8287 - acc: 0.6751 - val_loss: 1.9242 - val_acc: 0.3220 - lr: 0.0049
Epoch 5/100
50/50 [==============================] - 27s 549ms/step - batch: 24.5000 - size: 63.9200 - loss: 0.8004 - acc: 0.6824 - val_loss: 1.7102 - val_acc: 0.3352 - lr: 0.0048
Epoch 6/100
50/50 [==============================] - 28s 555ms/step - batch: 24.5000 - size: 63.8400 - loss: 0.7

In [21]:
test_path='/content/drive/MyDrive/Colab Notebooks/Test_Data/Test'
#size=224
images_names=[]
def create_test_data():
    testing_data=[]
    for img in os.listdir(test_path):
        path = os.path.join(test_path, img)
        img_data = cv2.imread(path,3)
        image_resize = cv2.resize(img_data,(size,size))
        image=cv2.cvtColor(image_resize, cv2.COLOR_BGR2RGB)
        testing_data.append([np.array(image), create_label(img)])
    shuffle(testing_data)
    np.save('testingsssss_data.npy', testing_data)
    return testing_data
if (os.path.exists('testingsssss_data.npy')):
    test_data =np.load('testingsssss_data.npy',allow_pickle=True)
else:
    test_data = create_test_data()

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [22]:

test = test_data
xtest = np.array([i[0] for i in test]).reshape(-3,size,size,3)
ytest = np.array([i[1] for i in test])
X_test=np.asarray(xtest).astype(np.float32)
y_test=np.asarray(ytest).astype(np.float32)
X_test = X_test / 255.0
mean = np.mean(X_test, axis=0)
X_test -= mean                  # Step 2
lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)


In [ ]:
score=model.evaluate(X_test,y_test)
print('Test Accuracy=',score[1])

Test Accuracy= 0.904325
